In [1]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
from collections import OrderedDict
import uproot
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/nfshome/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2

# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.18/00
3.6.8 (default, Apr 25 2019, 21:02:35) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]


In [2]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()
path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/V1p7/MC_Summer16/v3/'
bkg_path = path+"/bkg/wH/normalized/"
# greater than one lepton, lepton 27 GeV cut, greater than 2 jets
lumi = 137000 #in pb-1

# fpath_bkg['QCD'] = bkg_path+"QCD_HT50toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_1pb_weighted.root"
# fpath_bkg['DYJetsToLL'] = bkg_path+"DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_1pb_weighted.root"
# fpath_bkg['TTJets_DiLept'] = bkg_path+"TTJets_DiLept_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_1pb_weighted.root"
# fpath_bkg['TTJets_SingleLeptFromTbar'] = bkg_path+"TTJets_SingleLeptFromTbar_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_1pb_weighted.root"
# fpath_bkg['TTJets_SingleLeptFromT'] = bkg_path+"TTJets_SingleLeptFromT_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_1pb_weighted.root"
fpath_bkg['WJetsToLNu'] = bkg_path+"WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_1pb_weighted.root"
# fpath_bkg['WJetsToLNu_old'] = bkg_path+"WJetsToLNu_Pt-100ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_1pb_weighted.root"

# fpath_bkg['ZJetsToNuNu'] = bkg_path+"ZJetsToNuNu_Zpt-100toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_1pb_weighted.root"


In [ ]:
start_t = time.time()
if wH:
    sig_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/V1p7/MC_Summer16/v3/signals/wH/normalized/'
else:    
    sig_path = path+'/signals/zH/normalized/'

fpath_bkg['bbbb'] = sig_path + 'WH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root'
# fpath_sig['met+bb'] = sig_path + 'ppTohToSS1SS2_SS1Tobb_SS2Toveve_vh_withISR_mh125_mx50_pl1000_ev100000_1pb_weighted.root'
ntuples_path = '/storage/user/nasurijr/LLP_muon_sys_analysis/WH_bbbb/'

fpath_bkg['ntuples'] = ntuples_path+'WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_v7.root'
fpath_bkg['ntuples_wminus'] = ntuples_path+'WminusH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV_v7.root'
fpath_bkg['ntuples_wplus'] = ntuples_path+'WplusH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV_v7.root'

# ntuples_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonNtuple/V1p7/MC_Summer16/v3/sixie/'
# fpath_bkg['ntuples_wminus'] = ntuples_path+'WminusH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV-powheg-pythia8/Run2_displacedJetMuonNtupler_V1p7_MC_Summer16_RunIISummer16DR80Premix-PUMoriond17_80X_mcRun2_asymptotic_2016_TrancheIV_v6-v1_v3_v1/190825_024047/0000/displacedJetMuon_ntupler_1.root'
# fpath_bkg['ntuples_wplus'] = ntuples_path+'WplusH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV-powheg-pythia8/Run2_displacedJetMuonNtupler_V1p7_MC_Summer16_RunIISummer16DR80Premix-PUMoriond17_80X_mcRun2_asymptotic_2016_TrancheIV_v6-v1_v3_v1/190825_024414/0000/displacedJetMuon_ntupler_1.root'

NEvents = {}


for k,v in fpath_bkg.items():
    print (k, v)
#     tree[k] = rtnp.root2array(v)
    root_dir = uproot.open(v) 
    if k[:7] == 'ntuples':
        tree_bkg[k] = root_dir['ntuples']['llp']
        NEvents[k] = root_dir['ntuples']['NEvents'][1]

    else:
        tree_bkg[k] = root_dir['MuonSystem']
        NEvents[k] = root_dir['NEvents'][1]
    print(k,NEvents[k])
#     tree[k] = root_dir['ntuples']
#     a = tree_bkg[k]["weight"].array()
#     print ('weights',np.unique(a)*lumi)
#     print(len(a)/ NEvents[k])
#     print(a.shape)
print(time.time()-start_t)

WJetsToLNu /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/V1p7/MC_Summer16/v3//bkg/wH/normalized/WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_1pb_weighted.root
WJetsToLNu 259325900.0
bbbb /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/V1p7/MC_Summer16/v3/signals/wH/normalized/WH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root


In [ ]:
xsec = {}
xsec['ntuples_wminus'] = 5.328E-01*0.324
xsec['ntuples_wplus'] = 8.4E-01*0.324
xsec['ntuples'] = 61526.7
legend = {}
legend['WJetsToLNu'] = 'WJetsToLNu'
legend['bbbb'] = 'signal'
legend['ntuples_w'] ='Nathan signal'
legend['ntuples'] = 'Nathan WJetsToLNu'
print(xsec['ntuples_wminus'])

In [ ]:
nCsc = {}
weight = {}
nCsc['ntuples_w'] = []
weight['ntuples_w'] = []
for k,T in tree_bkg.items():
    start_t = time.time()
    if k[:9] == 'ntuples_w':
        ncsc_temp = list(T.array('nCsc'))
        nCsc['ntuples_w'] =  nCsc['ntuples_w']+ncsc_temp
        weight['ntuples_w'] = weight['ntuples_w']+[lumi*xsec[k]/NEvents[k]]*len(ncsc_temp)
        
        print(k,lumi*xsec[k]/NEvents[k])
    elif k[:7] == 'ntuples':
        nCsc[k] =T.array('nCsc')
        weight[k] =[lumi*xsec[k]/NEvents[k]]*len(nCsc[k])
        print(k,lumi*xsec[k]/NEvents[k])
    else:
        weight[k] = T.array('weight')*lumi
        nCsc[k] =T.array('nCsc')
        print(k,np.unique(weight[k]))
    print(time.time()-start_t)

# nCsc

In [ ]:
# Plotting ncsc
start_t = time.time()
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.94,0.85)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in nCsc.items():
#     print(v)
#     print(v)
#     print(v.shape)

    h[k] = create_TH1D( v, axis_title=['nCsc', 'Events'], name=k, binning=[50,0,500],weights = weight[k])
    print(k,np.sum(np.array(weight[k])[np.array(v)>=30]))
    print(len(v))
    print(np.count_nonzero(np.array(v)>=30))
    h[k].SetLineColor(std_color_list[i])
    if k[:7] == 'ntuples':
        h[k].SetLineStyle(2)
    if k == 'ntuples_w':
        h[k].SetLineColor(std_color_list[0])
    elif k == 'ntuples':
        h[k].SetLineColor(std_color_list[1])
    elif k == 'bbbb':
        h[k].SetLineColor(std_color_list[0])
    else:
        h[k].SetLineColor(std_color_list[1])

    leg.AddEntry(h[k], legend[k])
    h[k].SetMaximum(10E10)
    h[k].SetMinimum(1)
    h[k].Draw('same hist')
    i+=1
leg.Draw()


c.SetLogy()
c.Draw()
print(time.time()-start_t)

# nCsc with different hit vetoing

In [ ]:
start_t = time.time()
weight = {}
nCsc_recoJetVeto0p4 = {}
nCsc_recoJetVeto0p8 = {}
nCsc_recoJetVeto0p4_Me11Veto = {}
nCsc_recoJetVeto0p8_Me11Veto = {}
nCsc_recoJetVeto0p4_Me1112Veto = {}
nCsc_recoJetVeto0p8_Me1112Veto = {}
event_Me1112Veto = {}
event_recoJetVeto0p4 = {}
for k,T in tree_bkg.items():
    

        
    weight[k] = T.array('weight')*lumi
    nCsc[k] =T.array('nCsc')
    nCsc_recoJetVeto0p4[k] =T.array('nCsc_recoJetVeto0p4')
    nCsc_recoJetVeto0p8[k] =T.array('nCsc_recoJetVeto0p8')
    nCsc_recoJetVeto0p4_Me11Veto[k] =T.array('nCsc_recoJetVeto0p4_Me11Veto')
    nCsc_recoJetVeto0p8_Me11Veto[k] =T.array('nCsc_recoJetVeto0p8_Me11Veto')
    nCsc_recoJetVeto0p4_Me1112Veto[k] =T.array('nCsc_recoJetVeto0p4_Me1112Veto')
    nCsc_recoJetVeto0p8_Me1112Veto[k] =T.array('nCsc_recoJetVeto0p8_Me1112Veto')
    event_recoJetVeto0p4[k] = T.array('event_recoJetVeto0p4')
    event_Me1112Veto[k] = T.array('event_Me1112Veto')
print(time.time()-start_t)

In [ ]:
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.5,0.70,0.85,0.85)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
gr = {}
var = [event_recoJetVeto0p4, event_Me1112Veto,'AND']
name = ['event_recoJetVeto0p4','event_Me1112Veto','AND',]
# moment_xaxis = ['sig_et1','sig_et2','sig_e1','sig_e2','sig_e','sig_et','etD','eD']
legend = ['Eventveto recoJet #Delta R = 0.4','Event veto ME11 12','AND',]

qcd_key = 'WJetsToLNu'
sig_k = 'bbbb'
for i in range(len(var)):
    eff = {}
    thre = np.array(range(250))
    for k in tree_bkg.keys():
        if var[i] == 'AND':
            cond = np.logical_and(event_recoJetVeto0p4[k], event_Me1112Veto[k])
        else:
            cond = var[i][k]
        v = nCsc[k][cond]
        w = weight[k][cond]
        eff[k] = []
        for th in thre:
            eff[k].append(1.0*np.sum(w[v>th])/np.sum(weight[k]))
        eff[k] = np.array(eff[k])
#     Plotting ROC curve
    eff[sig_k] = eff[sig_k][np.logical_not(eff[qcd_key] == 0)]
    eff[qcd_key] = eff[qcd_key][np.logical_not(eff[qcd_key] == 0)]
    print(eff[qcd_key].shape, eff[sig_k].shape)
    gr[name[i]] = create_TGraph(np.reciprocal(eff[qcd_key]),eff[sig_k],axis_title = ['background rejection','#epsilon_{sig}'])
    gr[name[i]].SetLineColor(std_color_list[i])
    gr[name[i]].SetLineStyle(std_color_list[0])

    gr[name[i]].SetLineWidth(2)
#     gr[moment_xaxis[i]].SetMarkerColor(std_color_list[i])
#     gr[moment_xaxis[i]].SetMarkerStyle(std_color_list[i])


#     gr[name[i]].GetYaxis().SetRangeUser(0,0.3)
    gr[name[i]].GetYaxis().SetRangeUser(0.0, 1.0)
#         gr[name[i]].GetYaxis().SetRangeUser(1.0/np.max(eff['QCD']), 10000)


    leg.AddEntry(gr[name[i]],legend[i])

    gr[name[i]].Draw('ac' if i==0 else 'c')
#ROC curve
# c = rt.TCanvas('c','c', 800, 800)
# leg = rt.TLegend(0.5,0.70,0.85,0.85)
# leg.SetTextSize(0.022)
# leg.SetBorderSize(0)

# leg.SetEntrySeparation(0.01)


# var = [nCsc_recoJetVeto0p4,nCsc_recoJetVeto0p8,nCsc_recoJetVeto0p4_Me11Veto,nCsc_recoJetVeto0p8_Me11Veto,
#         nCsc_recoJetVeto0p4_Me1112Veto,nCsc_recoJetVeto0p8_Me1112Veto]
# name = ['nCsc_recoJetVeto0p4','nCsc_recoJetVeto0p8','nCsc_recoJetVeto0p4_Me11Veto','nCsc_recoJetVeto0p8_Me11Veto',
#         'nCsc_recoJetVeto0p4_Me1112Veto','nCsc_recoJetVeto0p8_Me1112Veto']
# # moment_xaxis = ['sig_et1','sig_et2','sig_e1','sig_e2','sig_e','sig_et','etD','eD']
# legend = ['recoJetVeto #Delta R = 0.4','recoJetVeto #Delta R = 0.8','recoJetVeto #Delta R = 0.4, Me11Veto',
#           'recoJetVeto #Delta R = 0.8, Me11Veto','recoJetVeto #Delta R = 0.4, Me11 12Veto',
#           'recoJetVeto #Delta R = 0.8, Me11 12Veto']

var = [nCsc_recoJetVeto0p4,nCsc_recoJetVeto0p4_Me11Veto,
        nCsc_recoJetVeto0p4_Me1112Veto]
name = ['nCsc_recoJetVeto0p4','nCsc_recoJetVeto0p4_Me11Veto',
        'nCsc_recoJetVeto0p4_Me1112Veto',]
legend = ['recoJetVeto #Delta R = 0.4','recoJetVeto #Delta R = 0.4, Me11Veto',
          'recoJetVeto #Delta R = 0.4, Me11 12Veto',]

qcd_key = 'WJetsToLNu'
sig_k = 'bbbb'
for i in range(len(var)):
    eff = {}
    thre = np.array(range(250))
    for k in tree_bkg.keys():
        v = var[i][k]
        eff[k] = []
        for th in thre:
            eff[k].append(1.0*np.sum(weight[k][v>th])/np.sum(weight[k]))
        eff[k] = np.array(eff[k])
#     Plotting ROC curve
    eff[sig_k] = eff[sig_k][np.logical_not(eff[qcd_key] == 0)]
    eff[qcd_key] = eff[qcd_key][np.logical_not(eff[qcd_key] == 0)]
    print(eff[qcd_key].shape, eff[sig_k].shape)
    gr[name[i]] = create_TGraph(np.reciprocal(eff[qcd_key]),eff[sig_k],axis_title = ['background rejection','#epsilon_{sig}'])
#     gr[name[i]].SetLineColor(std_color_list[int(i/2)])
#     gr[name[i]].SetLineStyle(std_color_list[i%2])
    gr[name[i]].SetLineColor(std_color_list[i])
    gr[name[i]].SetLineStyle(std_color_list[1])
    gr[name[i]].SetLineWidth(2)


#     gr[name[i]].GetYaxis().SetRangeUser(0,0.3)
    gr[name[i]].GetYaxis().SetRangeUser(0.0, 1.0)
#         gr[name[i]].GetYaxis().SetRangeUser(1.0/np.max(eff['QCD']), 10000)


    leg.AddEntry(gr[name[i]],legend[i])

    gr[name[i]].Draw('c')
leg.Draw()
c.SetLogx()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')



# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')




In [ ]:
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.5,0.70,0.85,0.85)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
gr = {}


# var = [nCsc_recoJetVeto0p4,nCsc_recoJetVeto0p8,nCsc_recoJetVeto0p4_Me11Veto,nCsc_recoJetVeto0p8_Me11Veto,
#         nCsc_recoJetVeto0p4_Me1112Veto,nCsc_recoJetVeto0p8_Me1112Veto]
# name = ['nCsc_recoJetVeto0p4','nCsc_recoJetVeto0p8','nCsc_recoJetVeto0p4_Me11Veto','nCsc_recoJetVeto0p8_Me11Veto',
#         'nCsc_recoJetVeto0p4_Me1112Veto','nCsc_recoJetVeto0p8_Me1112Veto']
# # moment_xaxis = ['sig_et1','sig_et2','sig_e1','sig_e2','sig_e','sig_et','etD','eD']
# legend = ['recoJetVeto #Delta R = 0.4','recoJetVeto #Delta R = 0.8','recoJetVeto #Delta R = 0.4, Me11Veto',
#           'recoJetVeto #Delta R = 0.8, Me11Veto','recoJetVeto #Delta R = 0.4, Me11 12Veto',
#           'recoJetVeto #Delta R = 0.8, Me11 12Veto']

var = [nCsc_recoJetVeto0p4,nCsc_recoJetVeto0p4_Me11Veto,
        nCsc_recoJetVeto0p4_Me1112Veto]
name = ['nCsc_recoJetVeto0p4','nCsc_recoJetVeto0p4_Me11Veto',
        'nCsc_recoJetVeto0p4_Me1112Veto',]
legend = ['recoJetVeto #Delta R = 0.4','recoJetVeto #Delta R = 0.4, Me11Veto',
          'recoJetVeto #Delta R = 0.4, Me11 12Veto',]

qcd_key = 'WJetsToLNu'
sig_k = 'bbbb'
for i in range(len(var)):
    eff = {}
    thre = np.array(range(250))
    for k in tree_bkg.keys():
        v = var[i][k]
        eff[k] = []
        for th in thre:
            eff[k].append(1.0*np.sum(weight[k][v>th])/np.sum(weight[k]))
        eff[k] = np.array(eff[k])
#     Plotting ROC curve
    eff[sig_k] = eff[sig_k][np.logical_not(eff[qcd_key] == 0)]*len(weight[sig_k])/NEvents[sig_k]/0.075
    eff[qcd_key] = eff[qcd_key][np.logical_not(eff[qcd_key] == 0)]*len(weight[qcd_key])/NEvents[qcd_key]
    print(eff[qcd_key].shape, eff[sig_k].shape)
    gr[name[i]] = create_TGraph(np.reciprocal(eff[qcd_key]),eff[sig_k],axis_title = ['background rejection','#epsilon_{sig}'])
#     gr[name[i]].SetLineColor(std_color_list[int(i/2)])
#     gr[name[i]].SetLineStyle(std_color_list[i%2])
    gr[name[i]].SetLineColor(std_color_list[i])
    gr[name[i]].SetLineStyle(std_color_list[1])
    gr[name[i]].SetLineWidth(2)


#     gr[name[i]].GetYaxis().SetRangeUser(0,0.3)
    gr[name[i]].GetYaxis().SetRangeUser(0.0, 1.0)
#         gr[name[i]].GetYaxis().SetRangeUser(1.0/np.max(eff['QCD']), 10000)


    leg.AddEntry(gr[name[i]],legend[i])

    gr[name[i]].Draw('ac' if i==0 else 'c')
leg.Draw()
c.SetLogx()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')




# nCsc histogram

In [ ]:
start_t = time.time()
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.36,0.70,0.85,0.92)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
var = [nCsc,nCsc_recoJetVeto0p4,
        nCsc_recoJetVeto0p4_Me1112Veto]
name = ['nCsc','nCsc_recoJetVeto0p4',
        'nCsc_recoJetVeto0p4_Me1112Veto']
legend = ['','recoJetVeto #Delta R = 0.4',
         'recoJetVeto #Delta R = 0.4, Me11 12Veto']

# var = [nCsc_recoJetVeto0p4,nCsc_recoJetVeto0p8,nCsc_recoJetVeto0p4_Me11Veto,nCsc_recoJetVeto0p8_Me11Veto,
#         nCsc_recoJetVeto0p4_Me1112Veto,nCsc_recoJetVeto0p8_Me1112Veto]
# name = ['nCsc_recoJetVeto0p4','nCsc_recoJetVeto0p8','nCsc_recoJetVeto0p4_Me11Veto','nCsc_recoJetVeto0p8_Me11Veto',
#         'nCsc_recoJetVeto0p4_Me1112Veto','nCsc_recoJetVeto0p8_Me1112Veto']
# # moment_xaxis = ['sig_et1','sig_et2','sig_e1','sig_e2','sig_e','sig_et','etD','eD']
# legend = ['recoJetVeto #Delta R = 0.4','recoJetVeto #Delta R = 0.8','recoJetVeto #Delta R = 0.4, Me11Veto',
#           'recoJetVeto #Delta R = 0.8, Me11Veto','recoJetVeto #Delta R = 0.4, Me11 12Veto',
#           'recoJetVeto #Delta R = 0.8, Me11 12Veto']



max_y = 0
qcd_key = 'WJetsToLNu'
sig_k = 'bbbb'
for i in range(len(var)):
    for k in tree_bkg.keys():
#         cond = np.logical_and(event_Me1112Veto[k],event_recoJetVeto0p4[k])
        h[k+str(i)] = create_TH1D( var[i][k], axis_title=['nCsc', 'Events'], name=k, binning=[30,0,300],weights = weight[k])
        if k == 'bbbb':
            h[k+str(i)].SetLineColor(std_color_list[0])
        else:
            h[k+str(i)].SetLineColor(std_color_list[1])
        h[k+str(i)].SetLineStyle(i*3)
        currentMax=h[k+str(i)].GetBinContent(h[k+str(i)].GetMaximumBin())
        if currentMax > max_y:
            max_y = currentMax
            h[k+str(i)].SetMaximum(5*currentMax)
            h[k+str(i)].SetMinimum(1)
        h[k+str(i)].Draw("same hist")

#     gr[moment_xaxis[i]].SetMarkerColor(std_color_list[i])
#     gr[moment_xaxis[i]].SetMarkerStyle(std_color_list[i])


#     gr[name[i]].GetYaxis().SetRangeUser(0,0.3)
#         gr[name[i]].GetYaxis().SetRangeUser(1.0/np.max(eff['QCD']), 10000)


        leg.AddEntry(h[k+str(i)],k+" "+legend[i])

leg.Draw()
c.SetLogy()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')

print(time.time()-start_t)


In [ ]:
start_t = time.time()
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.30,0.75,0.85,0.92)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
var = [nCsc,nCsc_recoJetVeto0p4,
        nCsc_recoJetVeto0p4_Me1112Veto]
name = ['nCsc','nCsc_recoJetVeto0p4',
        'nCsc_recoJetVeto0p4_Me1112Veto']
legend = ['','recoJetVeto #Delta R = 0.4',
         'recoJetVeto #Delta R = 0.4, Me11 12Veto']

# var = [nCsc_recoJetVeto0p4,nCsc_recoJetVeto0p8,nCsc_recoJetVeto0p4_Me11Veto,nCsc_recoJetVeto0p8_Me11Veto,
#         nCsc_recoJetVeto0p4_Me1112Veto,nCsc_recoJetVeto0p8_Me1112Veto]
# name = ['nCsc_recoJetVeto0p4','nCsc_recoJetVeto0p8','nCsc_recoJetVeto0p4_Me11Veto','nCsc_recoJetVeto0p8_Me11Veto',
#         'nCsc_recoJetVeto0p4_Me1112Veto','nCsc_recoJetVeto0p8_Me1112Veto']
# # moment_xaxis = ['sig_et1','sig_et2','sig_e1','sig_e2','sig_e','sig_et','etD','eD']
# legend = ['recoJetVeto #Delta R = 0.4','recoJetVeto #Delta R = 0.8','recoJetVeto #Delta R = 0.4, Me11Veto',
#           'recoJetVeto #Delta R = 0.8, Me11Veto','recoJetVeto #Delta R = 0.4, Me11 12Veto',
#           'recoJetVeto #Delta R = 0.8, Me11 12Veto']



max_y = 0
qcd_key = 'WJetsToLNu'
sig_k = 'bbbb'
for k in tree_bkg.keys():
    cond = np.logical_and(event_recoJetVeto0p4[k], event_Me1112Veto[k])    
    h[k+'event'] = create_TH1D( nCsc[k][cond], axis_title=['nCsc', 'Events'], name=k, binning=[30,0,300],weights = weight[k][cond])
    h[k+'jetveto'] = create_TH1D( nCsc_recoJetVeto0p4_Me1112Veto[k], axis_title=['nCsc', 'Events'], name=k, binning=[30,0,300],weights = weight[k])

    if k == 'bbbb':
        h[k+'event'].SetLineColor(std_color_list[0])
        h[k+'jetveto'].SetLineColor(std_color_list[0])

    else:
        h[k+'event'].SetLineColor(std_color_list[1])
        h[k+'jetveto'].SetLineColor(std_color_list[1])

    h[k+'event'].SetLineStyle(0)
    h[k+'jetveto'].SetLineStyle(3)

#     currentMax=h[k+str(i)].GetBinContent(h[k+str(i)].GetMaximumBin())
#     if currentMax > max_y:
#         max_y = currentMax
    h[k+'event'].SetMaximum(5*10e6)
    h[k+'event'].SetMinimum(1)

#         h[k+str(i)].SetMinimum(1)
    h[k+'event'].Draw("same hist")
    h[k+'jetveto'].Draw("same hist")
#     gr[moment_xaxis[i]].SetMarkerColor(std_color_list[i])
#     gr[moment_xaxis[i]].SetMarkerStyle(std_color_list[i])


#     gr[name[i]].GetYaxis().SetRangeUser(0,0.3)
#         gr[name[i]].GetYaxis().SetRangeUser(1.0/np.max(eff['QCD']), 10000)


    leg.AddEntry(h[k+'event'],k+" event veto #Delta R = 0.4, Me11 12Veto")
    leg.AddEntry(h[k+'jetveto'],k+" recoJetVeto #Delta R = 0.4, Me11 12Veto")

leg.Draw()
c.SetLogy()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')

print(time.time()-start_t)


# event yield vs. ncsc

In [ ]:
start_t = time.time()
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.45,0.80,0.85,0.92)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)
h = {}
leg.SetEntrySeparation(0.01)
var = [nCsc,nCsc_recoJetVeto0p4,
        nCsc_recoJetVeto0p4_Me1112Veto]
name = ['nCsc','nCsc_recoJetVeto0p4',
        'nCsc_recoJetVeto0p4_Me1112Veto']
legend = ['','recoJetVeto #Delta R = 0.4',
         'recoJetVeto #Delta R = 0.4, Me11 12Veto']
qcd_key = 'WJetsToLNu'
sig_k = 'bbbb'
threshold = np.array(list(range(300)))
for i in range(len(var)):
    event_count = {}
    for k in tree_bkg.keys():
        event_count[k] = []
        for th in threshold:
            event_count[k].append(np.sum(weight[k][var[i][k]>th]))
        event_count[k] = np.array(event_count[k])
    cond = np.logical_not(event_count[k] == 0)
#     gr['bbbb'] = create_TGraph(eff_bkg,eff_sig,axis_title = ['#epsilon_{bkg}','#epsilon_{signal}'])
    h[sig_k+str(i)] = create_TGraph(threshold[cond], event_count[sig_k][cond],  axis_title=['nCsc', 'Signal Yield'])
    h[sig_k+str(i)].SetLineColor(std_color_list[i])
    h[sig_k+str(i)].SetLineWidth(2)
    h[sig_k+str(i)].SetMarkerColor(std_color_list[i])
    h[sig_k+str(i)].GetXaxis().SetRangeUser(0, 300)
    h[sig_k+str(i)].SetMaximum(10000)

#     h[sig_k+str(i)].GetYaxis().SetRangeUser(0, 50000)
    leg.AddEntry(h[sig_k+str(i)],sig_k+" "+legend[i])
    h[sig_k+str(i)].Draw('ac' if i == 0 else 'c')
leg.Draw()

c.SetLogy()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')

print(time.time()-start_t)


# significance

In [ ]:
start_t = time.time()
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.35,0.80,0.85,0.92)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
var = [nCsc,nCsc_recoJetVeto0p4,
        nCsc_recoJetVeto0p4_Me1112Veto]
name = ['nCsc','nCsc_recoJetVeto0p4',
        'nCsc_recoJetVeto0p4_Me1112Veto']
legend = ['','recoJetVeto #Delta R = 0.4',
         'recoJetVeto #Delta R = 0.4, Me11 12Veto']
bkg_k = 'WJetsToLNu'
sig_k = 'bbbb'
br = 1

threshold = list(range(300))
threshold = np.array(list(range(150)))*2
print(threshold)
for i in range(len(var)):
    event_count = {}
    
    for k in tree_bkg.keys():
        event_count[k] = []
        for th in threshold:
            event_count[k].append(np.sum(weight[k][var[i][k]>th]))
        event_count[k] = np.array(event_count[k])
        if k == sig_k:
            event_count[k] = event_count[k]*br
    sig = event_count[sig_k]/np.sqrt(event_count[sig_k]+event_count[bkg_k])
    cond = event_count[sig_k]>0
    sig = sig[cond]
    ncsc = threshold[cond]
#     gr['bbbb'] = create_TGraph(eff_bkg,eff_sig,axis_title = ['#epsilon_{bkg}','#epsilon_{signal}'])
    h[sig_k+str(i)] = create_TGraph(ncsc, sig,  axis_title=['nCsc', 'Significnace'])
    h[sig_k+str(i)].SetLineWidth(2)
    h[sig_k+str(i)].SetMarkerColor(std_color_list[i])
    h[sig_k+str(i)].SetLineColor(std_color_list[i])
    h[sig_k+str(i)].GetXaxis().SetRangeUser(0, 300)
    h[sig_k+str(i)].GetYaxis().SetRangeUser(0, 15)


    leg.AddEntry(h[sig_k+str(i)],sig_k+" "+legend[i])
    h[sig_k+str(i)].Draw('ac' if i == 0 else 'c')

#     h[sig_k+str(i)].Draw('ac' if i==0 else 'c')
#     gr['bbbb'].Draw('')




leg.Draw()
# c.SetLogy()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')

print(time.time()-start_t)


# Reco Time vs. Gen Time

# Time response in rechit cut

# Time response in ECAL energy bins

# Time response in t_gen